In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd


from mpl_toolkits.axes_grid1 import make_axes_locatable
import time

import preseason.tools as sf
import preseason.onset_demise as od
import preseason.plotting as pp


In [ ]:
### Allows us to use dask to speed up some calculations ###
from dask.distributed import Client
client = Client()
client

In [ ]:
### Selecting out Peru from the global data.
lat_radius = 40 # degrees lat/lon
lon_radius = 50
PERU_center = {'lat': slice(0+lat_radius, -20-lat_radius), 'lon': slice(277-lon_radius, 293+lon_radius)}


In [ ]:
hgt_data_dir = '/data/deluge/reanalysis/REANALYSIS/ERA5/3D/daily/hgt/' 

ds_hgt = xr.open_mfdataset(hgt_data_dir+'*.nc',parallel=True, chunks={'latitude': 25, 'longitude': 25, 'time': -1})

hgt_data = ds_hgt['hgt'].sel(level=500, lat = PERU_center['lat'], lon = PERU_center['lon']) 

In [ ]:
hgt_anom = hgt_data - hgt_data.mean(dim='time')

In [ ]:
hgt_anom.compute()

In [ ]:
moisture_data_dir = '/data/deluge/reanalysis/REANALYSIS/ERA5/3D/daily/q/'

ds_q = xr.open_mfdataset(moisture_data_dir+'*.nc',parallel=True, chunks={'latitude': 25, 'longitude': 25, 'time': -1})

q_data = ds_q['q'].sel(level=500, lat = PERU_center['lat'], lon = PERU_center['lon']) 

In [ ]:
precip_data_dir = '/data/deluge/reanalysis/REANALYSIS/ERA5/2D/daily/precip/'

ds_p = xr.open_mfdataset(precip_data_dir+'*.nc',parallel=True, chunks={'latitude': 25, 'longitude': 25, 'time': -1})

precip_data = ds_p['precip'].sel(latitude = PERU_center['lat'], longitude = PERU_center['lon'])

In [ ]:
precip_data

In [ ]:
precip_data = precip_data.compute()

In [ ]:
precip_anom = precip_data - precip_data.mean(dim='time')


In [ ]:
p_annual_cycle = sf.calc_annual_cycle(precip_data)

In [ ]:
analysis_start = od.B17_analysis_start(p_annual_cycle)

In [ ]:
onset = od.onset_LM01(precip_anom, analysis_start)

In [ ]:
demise = od.demise_LM01(precip_anom, analysis_start)

In [ ]:
center_loc_onset = onset.sel(latitude=-10, longitude=285)
center_loc_demise = demise.sel(latitude=-10, longitude=285)

In [ ]:
early_onset = center_loc_onset.where(center_loc_onset < center_loc_onset.quantile(0.25))
late_onset = center_loc_onset.where(center_loc_onset > center_loc_onset.quantile(0.75))

In [ ]:
early_demise = center_loc_demise.where(center_loc_demise < center_loc_demise.quantile(0.25))
late_demise = center_loc_demise.where(center_loc_demise > center_loc_demise.quantile(0.75))

In [ ]:
early_onset_dates = sf.calcDates(early_onset).dropna(dim='year')
late_onset_dates = sf.calcDates(late_onset).dropna(dim='year')

In [ ]:
early_demise_dates = sf.calcDates(early_demise).dropna(dim='year')
late_demise_dates = sf.calcDates(late_demise).dropna(dim='year')

In [ ]:
early_dates.mean()

In [ ]:
late_dates.mean()

In [ ]:
composites_early_onset = create_composites(hgt_anom, early_onset_dates, period=4)
composites_late_onset = create_composites(hgt_anom, late_onset_dates, period=4)

In [ ]:
composites_early_demise = create_composites(hgt_anom, early_demise_dates, period=4)
composites_late_demise = create_composites(hgt_anom, late_demise_dates, period=4)

In [ ]:
pp.plot_spatial_data(composites_early_demise,vmin=-50, vmax=50, cmap='RdBu_r', var='Geopotential Anomaly', title='Early Demise Composite')

In [ ]:
pp.plot_spatial_data(composites_late_demise,vmin=-40, vmax=40, cmap='RdBu_r', var='Geopotential Anomaly', title='Late Demise Composite')

In [ ]:


def create_composites(ds, dates, period):
    composites = {}
    composite_data = []
    for date in dates:
        start_date = date - pd.Timedelta(period,'w')
        end_date = date - pd.Timedelta(1,"d")
        
        subset = ds.sel(time=slice(start_date, end_date))
        
        composite = subset.mean(dim='time')
        composite_data.append(composite)
    
    composites = xr.concat(composite_data, dim='time')
    composites = composites.mean(dim='time')
    return composites

# Create composites for each time period
#composites_1week = create_composites(ds, dates, period='1w')
#composites_2weeks = create_composites(ds, dates, period='2w')
#composites_3weeks = create_composites(ds, dates, period='3w')
#composites_1month = create_composites(ds, dates, period='1M')

In [ ]:
late_dates